<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER_Luigi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [1]:

!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint
from nltk.tokenize import word_tokenize

nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


Requirement already up-to-date: spacy[cuda92] in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 324.1MB 56kB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# ***Language Analysis***

In [0]:

"""
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …
"""

def get_info(sent_list):
  for sent in sent_list:
    print(sent)
    doc = nlp(sent)
    displacy.render(doc, style="dep", jupyter=True)  
    print("CHUNKS")
        
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.root.text, chunk.root.dep_, 
                chunk.root.head.text)
    print("TREE")
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])


def process(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    compounds = []
    for token in doc:
        if token.dep_ == "nsubjpass" and token.head.text in ["influenced", "influences"] :
            influenced.append(token.text)
        if token.dep_ == "nsubj" and token.head.text in ["influenced", "influences"]:
            influencers.append(token.text)
        if token.dep_ == "pobj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ in ["prep", "agent","conj"]:
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        if token.dep_ == "conj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ == "prep" or token.head.dep_ =="agent" or token.head.dep_ =="conj":
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        if token.dep_ == "dobj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ == "dobj" or token.head.dep_ == "prep" or token.head.dep_ =="agent":
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        
        if token.dep_ == "compound":
            compounds.append(token)
    
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers
    
phrases=["Mark was influenced by Jhon",
         "Jhon influenced Mark",
         "Mark was an influence of Jhon and Albert",
         "Jhon influenced him",
         "Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts ",
         "He is cited as an influence on John Milton"]

for phr in phrases:
  get_info([phr])
  print(process(phr))


Mark was influenced by Jhon


CHUNKS
Mark Mark nsubjpass influenced
Jhon Jhon pobj by
TREE
Mark nsubjpass influenced VERB []
was auxpass influenced VERB []
influenced ROOT influenced VERB [Mark, was, by]
by agent influenced VERB [Jhon]
Jhon pobj by ADP []
(['Mark'], ['Jhon'])
Jhon influenced Mark


CHUNKS
Jhon Jhon nsubj influenced
Mark Mark dobj influenced
TREE
Jhon nsubj influenced VERB []
influenced ROOT influenced VERB [Jhon, Mark]
Mark dobj influenced VERB []
([], ['Jhon', 'Mark'])
Mark was an influence of Jhon and Albert


CHUNKS
Mark Mark nsubj was
an influence influence attr was
Jhon Jhon pobj of
Albert Albert conj Jhon
TREE
Mark nsubj was AUX []
was ROOT was AUX [Mark, influence]
an det influence NOUN []
influence attr was AUX [an, of]
of prep influence NOUN [Jhon]
Jhon pobj of ADP [and, Albert]
and cc Jhon PROPN []
Albert conj Jhon PROPN []
([], [])
Jhon influenced him


CHUNKS
Jhon Jhon nsubj influenced
him him dobj influenced
TREE
Jhon nsubj influenced VERB []
influenced ROOT influenced VERB [Jhon, him]
him dobj influenced VERB []
([], ['Jhon', 'him'])
Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts 


CHUNKS
Adler Adler nsubjpass influenced
the philosophies philosophies pobj by
Immanuel Kant Kant pobj of
Friedrich Nietzsche Nietzsche conj Kant
Rudolf Virchow Virchow conj Nietzsche
the statesman statesman conj Virchow
Jan Smuts Smuts appos statesman
TREE
Adler nsubjpass influenced VERB []
was auxpass influenced VERB []
also advmod influenced VERB []
influenced ROOT influenced VERB [Adler, was, also, by]
by agent influenced VERB [philosophies]
the det philosophies NOUN []
philosophies pobj by ADP [the, of]
of prep philosophies NOUN [Kant]
Immanuel compound Kant PROPN []
Kant pobj of ADP [Immanuel, ,, Nietzsche]
, punct Kant PROPN []
Friedrich compound Nietzsche PROPN []
Nietzsche conj Kant PROPN [Friedrich, ,, Virchow]
, punct Nietzsche PROPN []
Rudolf compound Virchow PROPN []
Virchow conj Nietzsche PROPN [Rudolf, and, statesman]
and cc Virchow PROPN []
the det statesman NOUN []
statesman conj Virchow PROPN [the, Smuts]
Jan compound Smuts PROPN []
Smuts appos statesman NOUN [Jan]
(['

CHUNKS
He He nsubjpass cited
an influence influence pobj as
John Milton Milton pobj on
TREE
He nsubjpass cited VERB []
is auxpass cited VERB []
cited ROOT cited VERB [He, is, as]
as prep cited VERB [influence]
an det influence NOUN []
influence pobj as SCONJ [an, on]
on prep influence NOUN [Milton]
John compound Milton PROPN []
Milton pobj on ADP [John]
([], [])


#***Import JSON articles***


In [0]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            {
                "0": "name_of_someone_philosopher_influenced_by"
            }
        ]
    "table_influences":
        [
            {
                "0": "name_of_someone_philosopher_influences"
            }
        ]
}
"""
#open the file
with open('/content/drive/My Drive/Wiki_dump/mattia_ground_t.json') as f:
  jsonlist = json.load(f)

# ***Text Preprocessing***

In [0]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, -ing etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""

"""
import time
start = time.time()
counter_influence = []
counter_influenced = []
counter_influences = []
counter_influencing = []
counter_inspired = []
counter_inspire = []
counter_inspiration = []
for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing", "inspired", "inspire", "inspiration"]


    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if "influence" in word_list:
            counter_influence.append(word_list)
        if "influencing" in word_list:
            counter_influencing.append(word_list)
        if "influenced" in word_list:
            counter_influenced.append(word_list)
        if "influences" in word_list:
            counter_influences.append(word_list)
        if "inspired" in word_list:
            counter_inspired.append(word_list)
        if "inspire" in word_list:
            counter_inspire.append(word_list)
        if "inspiration" in word_list:
            counter_inspiration.append(word_list)
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list
print(time.time() - start)

"""
"""
influencing pisciato
inspire pisciato

influence only used as a name
influences only used as a name, plural of influences
inspiration only used as a name, similar to influence
influenced used as verb
inspired used like influenced

X's can be changed with his
Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X be Y's influence

influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y

influencing:
none

influenced
Y be influenced by X
X have influenced Y
Y's thinking was influenced by X
X influenced Y

X ispiratore Y ispirante
inspired
Y be influenced by X
X have inspired Y
Y had been inspired by X
Y's thinking/work was inspired by X
Y was inspired by X
X inspired Y

inspire
X seemed/(verb) to inspire Y
X's work inspire Y

inspiration
X became/was inspiration for Y
X's work provided/was inspiration for Y
X's work served as inspiration to Y
Y took/drew inspiration from X
X provided inspiration to Y
"""

print(len(counter_influence), "influence",
      len(counter_influences), "influences",
      len(counter_influencing), "influencing",
      len(counter_influenced), "influenced",
      len(counter_inspired), "inspired",
      len(counter_inspire), "inspire",
      len(counter_inspiration), "inspiration")
for wl in counter_inspired:
    print(' '.join(word for word in wl))

# ***Processing Giuliano(Spacy Dependencies)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""



"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
"""
import re
print(jsonlist_no_empty[20]["article"][0])
tokens = [i for i in jsonlist_no_empty[20]["article"][0] if i.isupper() ]
print(tokens)
"""

for phil in jsonlist_no_empty:
    display_info(phil["article"])




# ***Processing Mattia(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""

# ***Processing Luigi(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])

X ispiratore Y ispirante
inspired
Y be influenced by X
X have inspired Y
Y had been inspired by X
Y's thinking/work was inspired by X
Y was inspired by X
X inspired Y

inspire
X seemed/(verb) to inspire Y
X's work inspire Y

inspiration
X became/was inspiration for Y
X's work provided/was inspiration for Y
X's work served as inspiration to Y
Y took/drew inspiration from X
X provided inspiration to Y

"""


phrases_inspire=["He seemed to inspire her",
                    "These revolutionizing ideas of Wang Yangming would later inspire prominent Japanese thinkers like Motoori Norinaga",
                   "John's work inspire Mark",]
phrases_inspiration=["John's inspiration include him",
                   "John was inspiration for Mark",
                   "Rozanov is the main source of inspiration for Dmitry Galkovsky ",
                   "Jhon and Adam became inspiration for Mark",
                   "Jhon's work provided inspiration for Mark",
                   "He got the inspiration for this text from Schleiermacher ’ s Über die Religion ",
                   "Jhon's work was inspiration for Mark",
                   "While Murdoch 's thought is an inspiration for Conradi",
                   "Jhon's work served as inspiration to Mark",
                   "Lucian 's True Story inspired Cyrano de Bergerac , whose writings later served as inspiration for Jules Verne ",
                   "Mark took inspiration from John",
                   "He also took inspiration from phenomenologist epistemology",
                   "Mark drew inspiration from John",
                   "In particular , he drew inspiration from a Chinese Buddhist master named Tao-cho",
                   "Mark provided inspiration to John"]
phrases_inspired=["He was inspired by him",
                   "Mark have inspired Jhon",
                   "Jhon had been inspired by Marks",
                   "In it , Petrarch claimed to have been inspired by Philip V of Macedon",
                   "Jhon's thinking was inspired by Mark",
                   "Newton 's work on infinite series was inspired by Simon Stevin 's decimals",
                   "Jhon's work was inspired by Marks",
                   "Schiller was inspired by the play Julius of Tarent by Johann Anton Leisewitz .",
                   "Mark was inspired by Jhon",
                   "As a youth , he was inspired by Mencius ’ proposition",
                   "Jhon inspired Mark",
                   "It also inspired him to take falsifiability as his criterion of demarcation between what is , and is not , genuinely scientific",
                   "Jhon inspired Mark's work",
                   "Spinoza inspired the poet Shelley to write his essay"]
                   